<a href="https://colab.research.google.com/github/GustavoFortti/spark-high-performance/blob/master/Teste_Engenharia_de_Dados_v2_1_Gustavo_Fortti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [1]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 90 minutos.

# Python

In [3]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

**[DICA]**: Para realizar a atividade, pense em iterar por cada elemento tanto do dicionário quanto da lista de tuplas e fazer as comparações dentro das iterações.

In [292]:
def filtra_alunos_acima_corte(alunos, notas, nota_corte):
    # Desenvolva aqui

    ax_notas = {}
    for i in notas:
      if (notas[i] >= nota_corte): 
        ax_notas[i] = notas[i]

    ax_alunos = []
    for i in alunos:
      if (i[1] in ax_notas.keys()):
        ax_alunos.append((i[0], ax_notas[i[1]]))

    return sorted(ax_alunos, key=lambda name: name[0], reverse=False)
    
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)

[('Arthur', 6.3), ('Gabriella', 7.1), ('Maria', 9.5), ('Pedro', 8.7)]

# SQL

**Setup**


In [24]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   2201      0 --:--:-- --:--:-- --:--:--  2196
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  1565k      0 --:--:-- --:--:-- --:--:-- 1565k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   275k      0 --:--:-- --:--:-- --:--:--  275k


In [25]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [26]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

**Função para execução de queries**

In [27]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [40]:
q(
    """
      SELECT COUNT(v.cod_usuario) AS count_vendas_sp_mg
      FROM usuarios as u
      INNER JOIN vendas as v
      ON u.cod_usuario = v.cod_usuario
      WHERE estado in ("Minas Gerais", "São Paulo")
    """
)

+------------------+
|count_vendas_sp_mg|
+------------------+
|108               |
+------------------+



2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

**[DICA]**: perceba que a primeira parte da pergunta exige uma contagem do grão *usuário* e a segunda exige uma contagem do grão *vendas*.

In [95]:
q(
    """
        SELECT cod_usuario, count, estado, valor_total FROM (
          SELECT v.cod_usuario, COUNT(v.cod_usuario) AS count, estado, sum(valor) as valor_total, ROW_NUMBER() OVER(PARTITION by estado ORDER BY COUNT(v.cod_usuario) DESC) AS row
          FROM usuarios as u
          INNER JOIN vendas as v
          ON u.cod_usuario = v.cod_usuario
          GROUP BY v.cod_usuario, estado
        ) WHERE row = 1
    """
)

+-----------+-----+-------------------+------------------+
|cod_usuario|count|estado             |valor_total       |
+-----------+-----+-------------------+------------------+
|2186       |7    |Amapá              |51359.34          |
|2981       |9    |Bahia              |126282.4          |
|1787       |8    |Ceará              |121494.67         |
|700        |9    |Goiás              |142389.09         |
|2811       |13   |Maranhão           |184101.88999999998|
|2340       |9    |Mato Grosso        |165197.3          |
|679        |6    |Mato Grosso do Sul |152271.96         |
|3090       |9    |Minas Gerais       |95697.3           |
|2843       |10   |Paraná             |137228.01         |
|464        |6    |Paraíba            |73556.09999999999 |
|2646       |7    |Pernambuco         |135556.7          |
|2441       |5    |Piauí              |40866.189999999995|
|3403       |10   |Rio Grande do Norte|136347.00999999998|
|28         |9    |Rio Grande do Sul  |29114.44999999999

3) Quais são os usuários que não fizeram nenhuma compra?

In [141]:
q(
    """
        SELECT U.cod_usuario
        FROM usuarios as u
        LEFT JOIN vendas as v
        ON u.cod_usuario = v.cod_usuario
        WHERE v.cod_usuario is NULL
    """
)

+-----------+
|cod_usuario|
+-----------+
|5098       |
|5229       |
|5482       |
|5504       |
|5614       |
|5923       |
|6985       |
|7901       |
|9980       |
|10080      |
|10152      |
|10170      |
|10929      |
|11002      |
|11546      |
|12298      |
|12354      |
|13117      |
|13476      |
|14486      |
|15198      |
|15313      |
|15784      |
|16570      |
|17530      |
|17779      |
|18028      |
|18034      |
|18166      |
|18366      |
+-----------+
only showing top 30 rows



4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra *por faixa etária*?

In [159]:
q(
    """
        SELECT faixa_etaria, MEAN(valor) AS media, COUNT(v.cod_usuario) AS quantidade
        FROM usuarios as u
        LEFT JOIN vendas as v
        ON u.cod_usuario = v.cod_usuario
        GROUP BY faixa_etaria
    """
)

+------------------+------------------+----------+
|faixa_etaria      |media             |quantidade|
+------------------+------------------+----------+
|Entre 22 a 27 anos|12203.022968749996|64        |
|Entre 10 a 15 anos|8890.125348837211 |43        |
|Entre 16 a 21 anos|12725.947000000002|50        |
|Entre 50 a 61 anos|10849.47575       |80        |
|null              |null              |0         |
|Entre 37 a 49 anos|16727.190714285716|42        |
|Entre 62 a 70 anos|10012.626410256411|39        |
|Entre 28 a 36 anos|8233.037142857142 |56        |
|Mais de 70 anos   |11303.103793103446|29        |
+------------------+------------------+----------+



In [160]:
spark.stop()

# PySpark

**setup**:

In [161]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   5837      0 --:--:-- --:--:-- --:--:--  5837
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  2840k      0 --:--:-- --:--:-- --:--:-- 2855k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0  1050k      0 --:--:-- --:--:-- --:--:-- 1050k


In [162]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [174]:
import pyspark.sql.functions as F

In [163]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [205]:

vendas.select(['valor', 'quantidade']).agg(
    F.count(F.col('quantidade')).alias("count_compras"),
    F.sum(F.col('quantidade')).alias("total_itens"),
    F.sum(F.col('valor')).alias("total_receita")
).show()

+-------------+-----------+--------------------+
|count_compras|total_itens|       total_receita|
+-------------+-----------+--------------------+
|        20000|     209149|2.1584900165999958E8|
+-------------+-----------+--------------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [248]:
estados = ["Santa Catarina", "Rio Grande do Sul", "Minas Gerais", "São Paulo", "Rio de Janeiro", "Espirito Santo", "Paraná"]

usuarios = usuarios.alias("u")

df_produto = usuarios.join(vendas, usuarios.cod_usuario == vendas.cod_usuario) \
        .join(produtos, produtos.cod_produto == vendas.cod_produto) \
        .select(["nome_produto", "estado", "quantidade", 'valor', 'u.cod_usuario'])

df_produto[df_produto.estado.isin(estados)].groupby("nome_produto").agg(
    F.sum("valor").alias("total"), \
    F.mean("valor").alias("media_preco"), \
    F.sum("quantidade").alias("quantidade_vendida")
).show()

+--------------------+------------------+------------------+------------------+
|        nome_produto|             total|       media_preco|quantidade_vendida|
+--------------------+------------------+------------------+------------------+
|Bicicleta Caloi A...|          95550.38|15925.063333333334|                71|
| Cafeteira Nespresso|          13832.91|          1383.291|                63|
|Notebook Asus Int...| 615180.6000000001|43941.471428571436|               134|
|Aparelho de Barbe...|           10002.3|         1250.2875|                77|
|     SPA Intel 700 L| 719892.0000000001|  55376.3076923077|               120|
|Jogo Mortal Komba...|            4441.1|            444.11|                89|
|Umidificador de A...|            7130.0|            891.25|                92|
|       Blusa Lacoste|          17095.77|         1424.6475|               123|
|Alcool em Gel 70%...|1397.2000000000007|53.738461538461564|               280|
|Monitor LG 19 pol...| 56073.59999999999

3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles?

In [256]:
# df_produto.groupBy()
df_produto.groupBy('nome_produto').agg(
    F.count(F.col("cod_usuario")), \
    F.max(F.col("valor")), \
    F.min(F.col("valor"))
).show()

+--------------------+------------------+----------+----------+
|        nome_produto|count(cod_usuario)|max(valor)|min(valor)|
+--------------------+------------------+----------+----------+
|Bicicleta Caloi A...|                15|   26915.6|   2691.56|
| Cafeteira Nespresso|                21|   3732.69|    219.57|
|Notebook Asus Int...|                29|   82636.2|    4590.9|
|Aparelho de Barbe...|                13|    2598.0|     129.9|
|     SPA Intel 700 L|                27|  119982.0|    5999.1|
|Jogo Mortal Komba...|                18|     848.3|      49.9|
|Umidificador de A...|                21|    1550.0|      77.5|
|       Blusa Lacoste|                18|    2779.8|    138.99|
|Alcool em Gel 70%...|                36|      99.8|      4.99|
|Monitor LG 19 pol...|                28|   11097.9|     584.1|
|   Escrivaninha em L|                27|    8740.0|     874.0|
|Lenço umedecido T...|                13|     158.0|       7.9|
|Kit 3 barras de c...|                16

4) Construa uma tabela unificada com todas as colunas dos 3 datasets (sem colunas repetidas) e escreva essa tabela localmente em formato **parquet** particionando pelo estado.

In [278]:
usuarios = usuarios.alias("u")
vendas = vendas.alias("v")
produtos = produtos.alias("p")

df = usuarios.join(vendas, ['cod_usuario']) \
      .join(produtos, ['cod_produto'])

df.write.option("header",True) \
        .partitionBy("estado") \
        .mode("overwrite") \
        .parquet("data/")
  

# FIM!